## Build the structured data files from the raw data:

In [1]:
from fpl_predictor.build_data import DataBuilder

# Run this code to update the structured data:
db = DataBuilder()

# Run this to access the object without building files:
db = DataBuilder(build_player_files=False, build_master_csv=False)

# Example function:
db.load_year_players(2019).head()

Making master player files ... ✓
Making master player gameweek CSV file ... ✓


,name,code,element_type,first_name,id,second_name,squad_number,team,team_code,web_name
0,Shkodran Mustafi,69140,2,Shkodran,1,Mustafi,None,1,3,Mustafi
1,Héctor Bellerín,98745,2,Héctor,2,Bellerín,None,1,3,Bellerín
2,Sead Kolasinac,111457,2,Sead,3,Kolasinac,None,1,3,Kolasinac
3,Ainsley Maitland-Niles,154043,2,Ainsley,4,Maitland-Niles,None,1,3,Maitland-Niles
4,Sokratis Papastathopoulos,39476,2,Sokratis,5,Papastathopoulos,None,1,3,Sokratis


## Get information about players:

In [2]:
from fpl_predictor.player_information import PlayerInformation
pi = PlayerInformation()
pi.search_players("rui", "patr")

{38533: 'Rui Pedro dos Santos Patrício'}

In [3]:
# Get more details from a specific gameweek:
code = 38533
pi.get_player(code, year=2019, week=5)

{'code': 38533,
 'name': 'Rui Pedro dos Santos Patrício',
 'position': 'GK',
 'value': 51,
 'team': 'Wolves'}

## Build a Squad:

In [23]:
# Use this object to build a squad:
from fpl_predictor.squad_builder import SquadBuilder

# Scoring metric is the column from the data that you want to rank players by:
sqb = SquadBuilder()

In [5]:
codes = [38533, 39487, 149484, 206325, 172780, 103955, 118748, 61366, 44699, 
         57127, 193111, 98747, 109533, 80146, 153723]
sqb.squad.remove_all_players()
for code in codes:
    sqb.add_player(code=code, year=2019, week=5)

Player added to squad: Rui Pedro dos Santos Patrício
Player added to squad: Erik Pieters
Player added to squad: Tyrone Mings
Player added to squad: Oleksandr Zinchenko
Player added to squad: James Maddison
Player added to squad: Raheem Sterling
Player added to squad: Mohamed Salah
Player added to squad: Kevin De Bruyne
Player added to squad: Ashley Barnes
Player added to squad: Teemu Pukki
Player added to squad: Todd Cantwell
Player added to squad: Nick Pope
Player added to squad: Emerson Palmieri dos Santos
Player added to squad: Jordan Ayew
Player added to squad: John Lundstram


In [24]:
# Load a squad from CSV:
sqb.squad.remove_all_players()
sqb.squad.load_squad("gw10")

Squad of 15 players loaded from /Users/tp/projects/fpl-predictor/data/squads/gw10.csv


In [25]:
# See the first team chosen by the scoring_metric:
sqb.squad.selected

,code,name,position,value,score,score_per_value,team
0,103955,Raheem Sterling,MID,123,17.3,0.140650,Man City
1,118748,Mohamed Salah,MID,125,15.9,0.127200,Liverpool
2,85971,Heung-Min Son,MID,98,10.2,0.104082,Spurs
3,193111,Todd Cantwell,MID,48,9.2,0.191667,Norwich
4,57127,Teemu Pukki,FWD,70,9.0,0.128571,Norwich
5,173879,Tammy Abraham,FWD,77,8.6,0.111688,Chelsea
6,80146,Jordan Ayew,FWD,51,6.0,0.117647,Crystal Palace
7,171129,Diego Rico,DEF,42,5.8,0.138095,Bournemouth
8,93100,Jannik Vestergaard,DEF,52,5.4,0.103846,Southampton
9,39487,Erik Pieters,DEF,47,3.5,0.074468,Burnley


In [26]:
sqb.squad.substitutes

,code,name,position,value,score,score_per_value,team
0,184341,Mason Mount,MID,67,3.1,0.046269,Chelsea
1,153723,John Lundstram,DEF,42,2.5,0.059524,Sheffield Utd
2,149484,Tyrone Mings,DEF,46,1.8,0.039130,Aston Villa
3,38533,Rui Pedro dos Santos Patrício,GK,51,1.0,0.019608,Wolves


In [27]:
sqb.squad.first_team

,code,name,position,score,score_per_value,team,value
0,98747,Nick Pope,GK,3.1,0.067391,Burnley,46.0
1,103955,Raheem Sterling,MID,17.3,0.140650,Man City,123.0
2,118748,Mohamed Salah,MID,15.9,0.127200,Liverpool,125.0
3,85971,Heung-Min Son,MID,10.2,0.104082,Spurs,98.0
4,193111,Todd Cantwell,MID,9.2,0.191667,Norwich,48.0
5,57127,Teemu Pukki,FWD,9.0,0.128571,Norwich,70.0
6,173879,Tammy Abraham,FWD,8.6,0.111688,Chelsea,77.0
7,80146,Jordan Ayew,FWD,6.0,0.117647,Crystal Palace,51.0
8,171129,Diego Rico,DEF,5.8,0.138095,Bournemouth,42.0
9,93100,Jannik Vestergaard,DEF,5.4,0.103846,Southampton,52.0


## Captain & Vice-Captain Picks

In [28]:
sqb.squad.captain

{103955: 'Raheem Sterling'}

In [29]:
sqb.squad.vice_captain

{118748: 'Mohamed Salah'}

## Evaluate potential transfers:

In [18]:
# Evaluate potential transfers from the currently selected squad:
from fpl_predictor.strategies.sample import evaluate_transfers, make_tranfers

# Change the criteria for ranking players:
sqb.cross_seasons = True
sqb.set_n(5)
sqb.set_scoring_metric("ict_index")

# Get suggested transfers:
transfers = evaluate_transfers(sqb, 2019, 10, live=True)
transfers

,available,code,minutes_percent,name,out_code,out_name,out_score,out_score_per_value,out_team,out_value,position,score,score_per_value,team,value,score_gain,score_per_val_gain,val_gain,squad_unavailable,first_team_unavailable
0,1.0,171129.0,1.0,Diego Rico,206325.0,Oleksandr Zinchenko,4.8,0.085714,Man City,56.0,DEF,6.56,0.156190,Bournemouth,42.0,1.76,0.070476,14.0,True,True
1,1.0,114283.0,1.0,Jack Grealish,184341.0,Mason Mount,3.1,0.046269,Chelsea,67.0,MID,11.64,0.197288,Aston Villa,59.0,8.54,0.151019,8.0,False,False
2,1.0,171129.0,1.0,Diego Rico,149484.0,Tyrone Mings,1.8,0.039130,Aston Villa,46.0,DEF,6.56,0.156190,Bournemouth,42.0,4.76,0.117060,4.0,False,False
3,1.0,171129.0,1.0,Diego Rico,153723.0,John Lundstram,2.5,0.059524,Sheffield Utd,42.0,DEF,6.56,0.156190,Bournemouth,42.0,4.06,0.096667,0.0,False,False
4,1.0,171129.0,1.0,Diego Rico,39487.0,Erik Pieters,3.5,0.074468,Burnley,47.0,DEF,6.56,0.156190,Bournemouth,42.0,3.06,0.081722,5.0,False,False
5,1.0,131897.0,1.0,Mathew Ryan,38533.0,Rui Pedro dos Santos Patrício,1.0,0.019608,Wolves,51.0,GK,3.28,0.069787,Brighton,47.0,2.28,0.050179,4.0,False,False
6,1.0,114283.0,1.0,Jack Grealish,85971.0,Heung-Min Son,10.2,0.104082,Spurs,98.0,MID,11.64,0.197288,Aston Villa,59.0,1.44,0.093207,39.0,False,False
7,1.0,171129.0,1.0,Diego Rico,93100.0,Jannik Vestergaard,5.4,0.103846,Southampton,52.0,DEF,6.56,0.156190,Bournemouth,42.0,1.16,0.052344,10.0,False,False
8,1.0,131897.0,1.0,Mathew Ryan,98747.0,Nick Pope,3.1,0.067391,Burnley,46.0,GK,3.28,0.069787,Brighton,47.0,0.18,0.002396,-1.0,False,False


## Make Transfers:

In [19]:
sqb.squad.remove_player(206325)

Removed player from squad (14 remain): Oleksandr Zinchenko


In [21]:
sqb.add_player(code=171129, year=2019, week=9, live=True)

Player added to squad: Diego Rico


In [22]:
# Save a squad to CSV:
sqb.squad.save_squad("gw10")

Squad of 15 players saved at: /Users/tp/projects/fpl-predictor/data/squads/gw10.csv
